In [192]:
import tabula, camelot, os
import pandas as pd
import numpy as np

In [257]:
def convert_table(df):
    df.loc[-1] = df.columns
    df = df.sort_index()
    df.columns = np.arange(len(df.columns))
    return df.reset_index(drop=True)

def find_row(df, text):
    list_id = np.where((df[0].str.find(text) > 0) == True)[0]
    if len(list_id) == 0:
        return None
    else:
        return list_id[0] 
    
def get_vol_table_tabula(file_path):
    tables = tabula.read_pdf(file_path, pages="all", multiple_tables=True, silent=True)
    for table in range(len(tables)):
        df = pd.DataFrame(tables[table])
        df = convert_table(df)
        row_id = find_row(df, "期末発行済株式数(自己株式を含む)")
        if row_id is not None and len(df.columns) < 10 and len(df[0][0]) < 100:
            return df
    return None


In [259]:
def cut_vol_table(tables):
    for i in range(len(tables)):
        df = tables[i].df
        df_find_key = df[0].str.find('期末発行済株式数（自己株式を含む）')
        for i in df_find_key.index:
            if df_find_key[i] >= 0:
                id_have_key = i
                df = df.iloc[id_have_key:, :].reset_index(drop=True)
                return df

def convert_table_mix_data(df):
    df_add = df[3].str.split(' ', expand=True)
    df_add2 = df[4].str.split(' ', expand=True)
    for i in df.index:
        if df[2][i] == '':
            df[2][i] = df_add[0][i]
            df[3][i] = df_add[1][i]
        if df[5][i] == '':
            df[5][i] = df_add2[1][i]
            df[4][i] = df_add2[0][i]
    return df
     
def drop_empty_col(df):
    for col in df.columns:
        if (df[col]=='').all():
            df = df.drop(columns=col)
    df.columns = np.arange(len(df.columns))
    return df

def get_vol_table_camelot(file_path):
    tables = camelot.read_pdf(file_path, pages="all", multiple_tables=True, flavor="stream", suppress_stdout=True)
    df = cut_vol_table(tables)
    df = convert_table_mix_data(df)
    df = drop_empty_col(df)
    return df

In [260]:
def get_vol_table(file_path):
    df = get_vol_table_tabula(file_path)
    if df is None:
        df = get_vol_table_camelot(file_path)
    return df


In [261]:
for file in os.listdir():
    if file.endswith(".pdf"):
        print(file)
        df = get_vol_table(file)
        df.to_csv(file.replace('.pdf', '.docx'), index=False)

140120220131575670.pdf
140120230202599357_(1).pdf
5486-20080730-1400778F73.pdf
